# Algorithme de Grover

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import numpy as np

def grover_image(show_oracle=False, show_grover=False):

    fig, ax = plt.subplots(figsize=(5, 5))

    # Axes
    ax.arrow(0, 0, 1.2, 0, head_width=0.04, head_length=0.06, fc='black', ec='black', linewidth=1)
    ax.arrow(0, 0, 0, 1.2, head_width=0.04, head_length=0.06, fc='black', ec='black', linewidth=1)
    ax.text(1.3, 0, r'$|\text{états}\perp s\rangle$', fontsize=12, ha='left')
    ax.text(-0.05, 1.25, r'$|s\rangle$', fontsize=12, ha='right')

    # Initial state
    r = 1.0
    theta_0 = np.pi / 18
    x_init = r * np.cos(theta_0)
    y_init = r * np.sin(theta_0)
    ax.arrow(0, 0, x_init, y_init, head_width=0.04, head_length=0.06, 
            fc='blue', ec='blue', linewidth=1)
    ax.text(x_init+0.1, y_init, r'$|+\rangle$', fontsize=12, ha='left')

    # Arc indicating the angle
    arc_radius = 0.4
    theta_1 = 0
    theta_2 = theta_0
    arc = Arc((0, 0), width=2*arc_radius, height=2*arc_radius,
                angle=0, theta1=np.degrees(theta_1), theta2=np.degrees(theta_2),
                color='blue', lw=1.5)
    ax.add_patch(arc)

    # Label the angle
    theta_mid = (theta_1+theta_2)/2
    mid_x = (arc_radius) * np.cos(theta_mid)
    mid_y = (arc_radius) * np.sin(theta_mid)
    ax.text(mid_x*1.1, mid_y*1.1, r'$\theta$', color='blue', fontsize=12, ha='left', va='center')

    # Oracle
    if (show_oracle):
        x_oracle = x_init
        y_oracle = -y_init
        ax.arrow(0, 0, x_oracle, y_oracle, head_width=0.04, head_length=0.06, 
                fc='white', ec='blue', linewidth=1, linestyle='-')
        ax.text(x_init+0.1, -y_init, r'$U_f\,|+\rangle$', fontsize=12, ha='left')

    # Reflexion about |+>
    if (show_grover):
        theta_new = 3 * theta_0
        x_new = r * np.cos(theta_new)
        y_new = r * np.sin(theta_new)
        ax.arrow(0, 0, x_new, y_new, head_width=0.04, head_length=0.06, fc='green', ec='green', linewidth=1)
        ax.text(x_new+0.1, y_new, r'$G\,|+\rangle$', fontsize=12, ha='left')

        # Arc indicating the angle increment
        arc_radius = 0.6
        theta_1 = theta_0
        theta_2 = 3*theta_0
        arc = Arc((0, 0), width=2*arc_radius, height=2*arc_radius,
                    angle=0, theta1=np.degrees(theta_1), theta2=np.degrees(theta_2),
                    color='green', lw=1.5)
        ax.add_patch(arc)

        # Label the angle increment
        theta_mid = (theta_1+theta_2)/2
        mid_x = (arc_radius) * np.cos(theta_mid)
        mid_y = (arc_radius) * np.sin(theta_mid)
        ax.text(mid_x*1.1, mid_y*1.1, r'$2\theta$', color='green', fontsize=12, ha='left', va='center')

    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])

    for spine in ax.spines.values():
        spine.set_visible(False)

    plt.show()

## Introduction et concept d'Oracle

Problème de recherche, exemple:
- identifier une entrée dans une dase de données non-structurée
- identifier la solution d'un problème NP-complexe


Données possibles encodées sur $n$ bits: $x\in\{0,1\}^n$

On dispose d'une fonction $f$ permettant d'identifier si un état $x$ donné est la solution $s$:

- $f(x) = 1 \quad$ si $x=s$
- $f(x) = 0 \quad$ sinon

On appelle une telle fonction un "oracle"

Sur un ordinateur quantique, on encode les solutions possibles sur $n$ qubits ($N = 2^n$ valeurs possibles):

\begin{equation*}
    \ket {0\cdots 00}, \quad \ket {0\cdots 01}, \quad \ket {0\cdots 10}, \quad \cdots, \quad \ket {1\cdots 11}
\end{equation*}

On se sert de l'oracle pour construire une porte quantique que l'on peut insérer dans un circuit:

\begin{equation*}
    U_f \ket{x} = (-1)^{f(x)} \ket{x}
\end{equation*}

Cette porte logique "reconnaît" la solution du problème décrit par $f$ et a pour effet d'inverser le signe de la composante vectorielle dans la direction de la solution.

En pratique, une telle porte logique est implémentée en utilisant un qubit supplémentaire (*ancilla*), que nous pouvons changer de signe si l'oracle reconnaît $x$ comme solution: $(f(x)=1)$. Cette opération est souvent écrite en utilisant une opération XOR: 

\begin{equation*}
    \tilde U_f \ket{x}\ket{y} = \ket{x}\ket{y \oplus f(x)}
\end{equation*}

En initialisant le qubit $\ket y$ avec l'état $\ket- = \frac{1}{\sqrt 2}(\ket 0 - \ket 1)$, celui-ci reste inchangé par l'action de $\tilde U_f$ et nous construisons bien une porte équivalente à $U_f$,

\begin{equation*}
    \tilde U_f \ket{x}\ket{-} = (-1)^{f(x)} \ket{x}\ket{-}
\end{equation*}

En général, la discussion de ce genre d'algorithme se soucie peu de l'implémentation de l'oracle. Toutefois, comme le modèle de calcul quantique généralise le modèle classique, il est toujours possible en théorie d'implémenter le calcul de $f(x)$.

L'atout du modèle de calcul quantique se trouve dans la possibilité d'évaluer en parallèle toutes les valeurs possibles de $x$ qui peuvent être encodées dans l'état $\ket x$. Cela est réalisé en initialisant les qubits dans l'état $\ket +$, une superposition uniforme de toutes les valeurs possibles, par l'application d'une porte de Hadamard sur chaque qubit.

\begin{equation*}
    \ket{+} = H^{\otimes n} \ket{0 \cdots 00} = \frac{1}{\sqrt{N}} \ket{0 \cdots 00} + \frac{1}{\sqrt{N}} \ket{0 \cdots 01} + \cdots + \frac{1}{\sqrt{N}} \ket{s} + \cdots + \frac{1}{\sqrt{N}} \ket{1 \cdots 11}
\end{equation*}

Cet état initial étant un superposition uniforme des $N=2^n$ possibles, la probabilité de mesurer cet état dans l'état solution $\ket s$ n'est que de $1/N$.

L'action de l'oracle n'a pour effet que d'inverser le signe du coefficient de $\ket s$. Son rôle n'est que de marquer la composante dans la direction de la solution et, à lui seul, n'augmente pas la probabilité d'obtenir l'état solution.

\begin{equation*}
    U_f \ket{+} = \frac{1}{\sqrt{N}} \ket{0 \cdots 00} + \frac{1}{\sqrt{N}} \ket{0 \cdots 01} + \cdots - \frac{1}{\sqrt{N}} \ket{s} + \cdots + \frac{1}{\sqrt{N}} \ket{1 \cdots 11}
\end{equation*}

## Aspect géométrique

Nous pouvons visualiser plus simplement l'action de l'oracle en restreignant la représentation du système au plan sous-tendu par les deux vecteurs $\ket s$ et $\ket +$. En effet, nous pouvons regrouper les composantes de $\ket +$ perpendiculaire à $\ket s$ pour simplifier la notation:

\begin{equation*}
    \ket{+} = \frac{1}{\sqrt{N}} \ket{s} + \sqrt{1-\frac{1}{N}} \ket{x\perp s}
\end{equation*}

Et faire apparaître une interprétation géométrique en réécrivant les amplitudes avec des fonctions trigonométriques:

\begin{equation*}
    \ket{+} = \sin(\theta) \ket{s} + \cos(\theta) \ket{x\perp s}
\end{equation*}

Nous représenterons alors la situation sur un graphe où l'on place l'état solution $\ket{s}$ (inconnu) sur l'axe $y$ et l'espace de tous les autres états orthogonaux sur l'axe $x$. Le vecteur représentant l'état initial est incliné d'un petit angle $\theta \approx \sin(\theta) = 1/\sqrt{N}$ dans la direction de la solution.

In [ ]:
grover_image(show_oracle=True)

L'action de l'oracle consiste alors à changer le signe de cet angle $\theta$: 

\begin{equation*}
    U_f \ket{+} = -\sin(\theta) \ket{s} + \cos(\theta) \ket{x\perp s}
\end{equation*}

## Itération de Grover

Comme mentionné plus haut, l'oracle n'augmente pas la probabilité de mesurer notre état quantique dans l'état solution. La probabilité reste la même qu'au départ:

\begin{equation*}
    |\langle s|(U_f|+\rangle)|^2 = \sin^2(\theta) = \frac 1N = |\braket{s|+}|^2
\end{equation*}

Pour exploiter l'action de l'oracle et augmenter la probabilité d'obtenir la solution $\ket s$, nous alons réaliser une opération géométrique de réflexion autour de l'état initial $\ket{+}$, qui va rapprocher notre état quantique de $\ket{s}$.

L'application successive de l'oracle $U_f$ et de cette réflexion autour de $\ket{+}$ constitue une itération complète de l'algorithme de Grover, que nous représentons par l'opérateur $G$.

In [ ]:
grover_image(show_oracle=True, show_grover=True)

Algébriquement, cette opération est représentée par l'opérateur

\begin{equation*}
    U_+ = 2|+\rangle \langle+| - I
\end{equation*}

Son action sur un état $\ket\psi$ est bien d'inverser le signe de la composante perpendiculaire à $\ket +$:

\begin{equation*}
    \ket\psi = \alpha \ket{+} + \beta \ket{\psi_{\perp}}
\end{equation*}

\begin{align*}
    U_+ \ket\psi 
    &= 2|+\rangle \langle+ \ket\psi - I \ket\psi \\
    &= 2\alpha \ket{+} - (\alpha \ket{+} + \beta \ket{\psi_{\perp}}) \\
    &= \alpha \ket{+} - \beta \ket{\psi_{\perp}}
\end{align*}

Les applications successive de l'itération de Grover font à chaque fois tourner le vecteur d'un angle $2\theta$. La probabilité de mesurer l'état solution augmente donc progressivement. Après $k$ itérations, elle vaut

\begin{equation*}
    |\braket{s|\psi}|^2 = \sin^2((2k+1)\theta) \qquad k = 0,1,2,\cdots
\end{equation*}

Elle est maximale pour $k \approx \pi/4\theta$ itérations. L'algorithme de Grover nous permet donc de compléter le problème de recherche avec seulement $O(\sqrt{N})$ appels à l'oracle. (Mais attention, au-delà du nombre optimal d'itérations, la probabilité se met à décroître!)

![title](../figures/grover_full.png)

![title](../figures/grover_iter.png)

## Oracle qui identifie un état particulier

In [ ]:
from qiskit.quantum_info import Operator
from qiskit import QuantumCircuit
import numpy as np

def oracle(bitstring):

    n = len(bitstring)
    oracle_matrix = np.eye(2**n)
    index = int(bitstring, 2)
    oracle_matrix[index, index] = -1
    return Operator(oracle_matrix)

In [ ]:
etat_reconnu = "0010"

Uf = oracle(etat_reconnu)
display(Uf.draw("latex"))

n = len(etat_reconnu)
circuit_oracle = QuantumCircuit(n)
circuit_oracle.unitary(Uf, circuit_oracle.qubits, label='Uf')
circuit_oracle.draw('mpl')

## Circuit avec itérations de Grover

Sur un ordinateur quantique, $U_+$ est implémentée à l'aide de portes de Hadamard,

\begin{align*}
    U_+ 
    &= 2|+\rangle \langle+| - I \\
    &= 2 H^{\otimes n} |0\rangle \langle0| H^{\otimes n} - H^{\otimes n} H^{\otimes n} \\
    &= H^{\otimes n} (2|0\rangle \langle0| - I) H^{\otimes n} \\
    &= H^{\otimes n} U_0 H^{\otimes n}
\end{align*}

qui permettent d'opérer la réflexion géométrique autour de l'état $\ket{0\cdots 00}$, ce qui peut être implémenté au moyen d'une porte NOT controlée.

(Suggestion d'exercice: montrer que l'implémentation ci-dessous de $U_0$ renverse bien le signe des composante différentes de $\ket{0\cdots 00}$)

In [ ]:
# Premièrement, appliquer l'oracle
circuit_grover_iter = circuit_oracle.copy()

# Ensuite, appliquer les portes de hadamard
circuit_grover_iter.h(range(n))

# Puis, appliquer la réflexion géométrique autour de l'état |0...00>
circuit_grover_iter.x(range(n))
circuit_grover_iter.h(0)
circuit_grover_iter.mcx(list(range(1, n)),0)  # Porte NOT controlée sur le qubit 0
circuit_grover_iter.h(0)
circuit_grover_iter.x(range(n))

# Enfin, appliquer les portes de hadamard à nouveau
circuit_grover_iter.h(range(n))

circuit_grover_iter.draw('mpl')

In [ ]:
#from qiskit.circuit.library import grover_operator

#circuit_grover_iter = grover_operator(circuit_oracle)
#circuit_grover_iter.draw('mpl')
#circuit_grover_iter.decompose().draw('mpl')

In [ ]:
circuit = QuantumCircuit(n)

# Superposition initiale (état |+>)
circuit.h(range(n))

# Iterations de Grover
#num_iters = 1
num_iters = np.floor((np.pi/4)*np.sqrt(2**n))

for _ in range(int(num_iters)):
    circuit.append(circuit_grover_iter.to_gate(), range(n))

circuit.measure_all()
circuit.draw('mpl')

In [ ]:
### Too much noise: one grover iteration is better in this case
#from qiskit_ibm_runtime.fake_provider import FakeManilaV2
#backend = FakeManilaV2()

from qiskit_aer import AerSimulator
backend = AerSimulator()

In [ ]:
from qiskit.compiler import transpile

circuit_transpiled = transpile(circuit, backend, optimization_level=3)
#circuit_transpiled.draw('mpl')

In [ ]:
job = backend.run(circuit_transpiled, shots=1024)

result = job.result()

from qiskit.visualization import plot_histogram
plot_histogram(result.get_counts())